In [1]:
!pip install qiskit QuantumRingsLib quantumrings-toolkit-qiskit qiskit_aer

In [2]:
import QuantumRingsLib
from qiskit.circuit import QuantumCircuit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, AncillaRegister, transpile
#from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit, transpile
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from matplotlib import pyplot as plt
from qiskit_aer import Aer, AerSimulator
from qiskit.visualization import plot_histogram
import numpy as np
import math
from qiskit.visualization import plot_histogram
import fractions
import traceback
from time import time
from numpy import ceil, floor, gcd, log2, pi, zeros
from numpy.random import randint
from qiskit.circuit.quantumcircuit import Qubit, QubitSpecifier
from quantumrings.toolkit.qiskit import QrBackendV2
from quantumrings.toolkit.qiskit import QrJobV1

In [3]:
provider = QuantumRingsProvider(
    token='rings-200.74mVc6gEDkWjemhpBEZ1uaUbto6qMQrW',
    name='sudikin@iirsddn.ac.in'
)
backend = provider.get_backend("scarlet_quantum_rings")
shots = 1024

provider.active_account()

{'name': 'sudikin@iirsddn.ac.in',
 'token': 'rings-200.74mVc6gEDkWjemhpBEZ1uaUbto6qMQrW',
 'max_qubits': '200'}

In [ ]:
def qft(circuit, qreg, n, with_swaps=True):
    for j in range(n - 1, 0, -1):
        circuit.h(qreg[j])
        for k in range(j - 1, -1, -1):
            if (pi / float(2 ** (j - k))) > 0:
                circuit.cp(pi / float(2 ** (j - k)), qreg[j], qreg[k])
        if with_swaps:
            circuit.barrier()

    if with_swaps:
        for j in range(n // 2):
            circuit.swap(qreg[j], qreg[n - j - 1])
    return circuit


def qft_dagger(circuit, qreg, n, with_swaps=True):
    if with_swaps:
        for j in range(n // 2):
            circuit.swap(qreg[j], qreg[n - j - 1])

    for j in range(n):
        circuit.h(qreg[j])
        if j != n - 1:
            for k in range(j, -1, -1):
                if (pi / float(2 ** ((j + 1) - k))) > 0:
                    circuit.cp(-pi / float(2 ** ((j + 1) - k)), qreg[j + 1], qreg[k])
        if with_swaps:
            circuit.barrier()
    return circuit

# def controlled_mult_mod_N(
#     circuit: QuantumCircuit,
#     qreg: QuantumRegister,
#     a: int,
#     N: int,
#     n: int,
#     control_reg: QuantumRegister | Qubit,
#     ancilla_reg: QuantumRegister,
# ):
#     """Controlled version of the multiplication by a mod N"""
#     qft(circuit, ancilla_reg, n + 1, False)
#     for i in range(n):
#         controlled_phi_adder_mod_N(
#             circuit,
#             ancilla_reg,
#             (2**i) * a % N,
#             N,
#             n + 1,
#             [qreg[i], control_reg],
#             ancilla_reg[n + 1],
#         )
#     qft_dagger(circuit, ancilla_reg, n + 1, False)

#     for i in range(n):
#         circuit.cswap(control_reg, qreg[i], ancilla_reg[i])

#     a_inv = pow(a, -1, N)
#     qft(circuit, ancilla_reg, n + 1, False)

#     for i in range(n):
#         controlled_phi_adder_mod_N_inv(
#             circuit,
#             ancilla_reg,
#             pow(2, i) * a_inv % N,
#             N,
#             n + 1,
#             [qreg[i], control_reg],
#             ancilla_reg[n + 1],
#         )
#     qft_dagger(circuit, ancilla_reg, n + 1, False)

def controlled_phi_adder(
    circuit: QuantumCircuit,
    qreg: QuantumRegister,
    a: int,
    N: int,
    control_qubits: QuantumRegister | Qubit | list[Qubit],
    inverse: bool = False,
):
    """Controlled version of the phi_adder"""
    angles = get_angles(a, N)
    for i in range(N):
        if len(angles) == 0:
            break
        if not inverse:
            if (
                isinstance(control_qubits, QuantumRegister)
                or isinstance(control_qubits, Qubit)
                or len(control_qubits) == 1
            ):
                circuit.cp(
                    angles[i],
                    (
                        control_qubits
                        if isinstance(control_qubits, QuantumRegister)
                        or isinstance(control_qubits, Qubit)
                        else control_qubits[0]
                    ),
                    qreg[i],
                )

            else:
                cc_phase_gate(circuit, control_qubits, qreg[i], angles[i])
        else:
            if (
                isinstance(control_qubits, QuantumRegister)
                or isinstance(control_qubits, Qubit)
                or len(control_qubits) == 1
            ):
                circuit.cp(
                    -angles[i],
                    (
                        control_qubits
                        if isinstance(control_qubits, QuantumRegister)
                        or isinstance(control_qubits, Qubit)
                        else control_qubits[0]
                    ),
                    qreg[i],
                )
            else:
                cc_phase_gate(circuit, control_qubits, qreg[i], -angles[i])

def controlled_phi_adder_mod_N(
    circuit: QuantumCircuit,
    qreg: QuantumRegister,
    a: int,
    N: int,
    n: int,
    control_qubits: QuantumRegister | Qubit | list[Qubit],
    ancilla_qubit: Qubit,
):
    """Controlled version of the phi_adder"""
    controlled_phi_adder(circuit, qreg, a, n, control_qubits, False)
    phi_adder(circuit, qreg, N, n, True)
    qft_dagger(circuit, qreg, n, False)
    circuit.cx(qreg[n - 1], ancilla_qubit)
    qft(circuit, qreg, n, False)
    controlled_phi_adder(circuit, qreg, N, n, control_qubits, False)

    controlled_phi_adder(circuit, qreg, a, n, control_qubits, True)
    qft_dagger(circuit, qreg, n, False)
    circuit.x(qreg[n - 1])
    circuit.cx(qreg[n - 1], ancilla_qubit)
    circuit.x(qreg[n - 1])
    qft(circuit, qreg, n, False)
    controlled_phi_adder(circuit, qreg, a, n, control_qubits, False)

def get_angles(a, N):
    """Compute the angles for the QPE"""
    s = bin(int(a))[2:].zfill(N)
    angles = zeros([N])
    for i in range(0, N):
        for j in range(i, N):
            if s[j] == "1":
                angles[N - i - 1] += pow(2, -(j - i))
        angles[N - i - 1] *= pi
    return angles

def phi_adder(
    circuit: QuantumCircuit,
    qreg: QuantumRegister,
    a: int,
    N: int,
    inverse: bool = False,
):
    """Create the circuit that performs the addition by a in the Fourier space (mod N)"""
    angles = get_angles(a, N)
    for i in range(N):
        if len(angles) == 0:
            break
        if not inverse:
            circuit.p(angles[i], qreg[i])
        else:
            circuit.p(-angles[i], qreg[i])


def cc_phase_gate(
    circuit: QuantumCircuit,
    control_qubits: list[QubitSpecifier | Qubit | QuantumRegister],
    target_qubit: int,
    angle: float,
):
    """Create a doubly controlled phase gate"""
    if len(control_qubits) != 2:
        raise ValueError(
            "The doubly controlled phase gate needs exactly 2 control qubits"
        )

    circuit.cp(angle / 2, control_qubits[0], target_qubit)
    circuit.cx(control_qubits[1], control_qubits[0])
    circuit.cp(-angle / 2, control_qubits[0], target_qubit)
    circuit.cx(control_qubits[1], control_qubits[0])
    circuit.cp(angle / 2, control_qubits[1], target_qubit)

def controlled_phi_adder_mod_N_inv(
    circuit: QuantumCircuit,
    qreg: QuantumRegister,
    a: int,
    N: int,
    n: int,
    control_qubits: QuantumRegister | Qubit | list[Qubit],
    ancilla_qubit: Qubit,
):
    """Controlled version of the inverse phi_adder"""
    controlled_phi_adder(circuit, qreg, a, n, control_qubits, True)
    qft_dagger(circuit, qreg, n, False)
    circuit.x(qreg[n - 1])
    circuit.cx(qreg[n - 1], ancilla_qubit)
    circuit.x(qreg[n - 1])
    qft(circuit, qreg, n, False)
    controlled_phi_adder(circuit, qreg, a, n, control_qubits, False)

    controlled_phi_adder(circuit, qreg, N, n, control_qubits, False)
    qft_dagger(circuit, qreg, n, False)
    circuit.cx(qreg[n - 1], ancilla_qubit)
    qft(circuit, qreg, n, False)
    phi_adder(circuit, qreg, N, n, False)
#     controlled_phi_adder(circuit, qreg, a, n, control_qubits, True)

def qpe_period_finding(N, a, n):
    """Quantum Phase Estimation for period finding"""
    q_up = QuantumRegister(2 * n, "q_up")
    q_down = QuantumRegister(n, "q_down")
    a_q = QuantumRegister(n + 2, "a_q")
    c = ClassicalRegister(2 * n, "c")
    qpe = QuantumCircuit(q_up, q_down, a_q, c)
    try:
        # Initialize qubits
        qpe.h(q_up)
        qpe.x(q_down[0])

        # Apply controlled multiplication gates
        for i in range(n):
            controlled_mult_mod_N(
                qpe, q_down, int(pow(a, pow(2, i))), N, n, q_up[i], a_q
            )

        # Apply inverse QFT
        qft_dagger(qpe, q_up, 2 * n, True)

        # Measure the qubits
        qpe.measure(q_up, c)

        # Simulate the QuantumCircuit

        mybackend = QrBackendV2(provider, num_qubits = qpe.num_qubits)
        transpiled_qpe = transpile(qpe, mybackend)
        job = mybackend.run(transpiled_qpe, shots=1024, memory=True)
        result = job.result()
        mem = result.get_memory()
        counts = result.get_counts()
        plot_histogram(counts)

        # Extract the phase
        phase = int(mem[0], 2) / (2 ** (2 * n))

        # Find the period
        r = fractions.Fraction(phase).limit_denominator(N - 1).denominator

        return r

    except Exception as e:
        #print(f"Error: {e}")
        return "Error"


def shor():
    """Shor's algorithm with user input"""
    N = int(input("Enter the number to factorize (N): "))
    n = int(ceil(log2(N)))

    while True:
        a = randint(2, N - 1)
        if gcd(a, N) != 1:
            print(f"Lucky guess: {gcd(a, N)} is a factor of {N}")
            return gcd(a, N), N // gcd(a, N)

        print(f"The coprime of {N} is {a}")

        # Find the period
        r = qpe_period_finding(N, a, n)
        if r == "Error" or r % 2 != 0 or pow(a, r // 2, N) == 1:
            continue

        # Guesses for factors
        guesses = [gcd(pow(a, r // 2, N) - 1, N), gcd(pow(a, r // 2, N) + 1, N)]
        for guess in guesses:
            if guess not in [1, N] and (N % guess) == 0:
                return guess, N // guess


if __name__ == "__main__":
    print("Shor's Quantum Algorithm")
    start = time()
    factors = shor()
    end = time()
    print(f"The prime factors are {factors}: found in {end - start} seconds")


Streaming output truncated to the last 5000 lines.
The coprime of 862463409547 is 697055750423
The coprime of 862463409547 is 576788731154
The coprime of 862463409547 is 220548880816
The coprime of 862463409547 is 845469523780
The coprime of 862463409547 is 182209819559
The coprime of 862463409547 is 392606961122
The coprime of 862463409547 is 674322114994
The coprime of 862463409547 is 250884941911
The coprime of 862463409547 is 492831377299
The coprime of 862463409547 is 389030201636
The coprime of 862463409547 is 234502881647
The coprime of 862463409547 is 617733070475
The coprime of 862463409547 is 152445738108
The coprime of 862463409547 is 147693517863
The coprime of 862463409547 is 181399238670
The coprime of 862463409547 is 689560966629
The coprime of 862463409547 is 550240325618
The coprime of 862463409547 is 592782652536
The coprime of 862463409547 is 97507759204
The coprime of 862463409547 is 723276450224
The coprime of 862463409547 is 801923310928
The coprime of 86246340954